# Data Formatting and Preprocessing

### Methadology:
1. Basic formatting
2. Calculate active users by date
3. Calculate total bandwidth usage by date
4. Remove unnecessary JSON fields and bugged strings from subuser names
5. Parsing JSON and CSV files to data match between subuser and customer information using the fuzzywuzzy library

### 1. Basic formatting

Selects all columns from the initial 'subuser_bw_usage.csv' file and orders everything based on the Subuser, Date, and Bandwidth Usage columns.

In [ ]:
SELECT * FROM subuser_bw_usage.csv
ORDER BY Subuser, Date, `Bandwidth Usage`;


Converts 'Bandwidth Usage' column from mebibytes to gigabytes

In [ ]:
import pandas as pd

# Read subuser_bw_usage.csv
subuser_bw = pd.read_csv('subuser_bw_usage.csv')

# Create a copy of the CSV file with the converted 'Bandwidth Usage' column
new_subuser_bw = subuser_bw.copy()
new_subuser_bw['Bandwidth Usage'] = subuser_bw['Bandwidth Usage'] * 0.0009765625  # Convert from MiB to GiB

# Save the converted data to a new CSV file
new_subuser_bw.to_csv('subuser_bw_usage_sorted.csv', index=False)

### 2. Calculate active users by date

Converts the 'Date' column to datetime format, counts unique emails by date, and saves the results to a new CSV file.

In [ ]:
import pandas as pd

# Read the subuser_bw_usage_sorted.csv file
df = pd.read_csv('subuser_bw_usage_sorted.csv')

# Convert the 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Group the data by 'Date' and count the unique emails
unique_emails_by_date = df.groupby('Date')['Subuser'].nunique()

# Print the number of unique emails for each date
for date, count in unique_emails_by_date.items():
    print(f"Date: {date.date()}, Active Users: {count}")

# Save the result to a new CSV file
unique_emails_by_date.to_csv('active_users_by_date.csv', header=True)


### 3. Calculate total bandwidth usage by date 

Groups data by the 'Date' column using the groupby() method and calculates the total bandwidth usage into a new CSV file with the to_csv() function.

In [ ]:
import pandas as pd

# Read the subuser_bw_usage_sorted.csv file
df = pd.read_csv('subuser_bw_usage_sorted.csv')

# Convert the 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Group the data by 'Date' and calculate the total bandwidth usage
usage_by_date = df.groupby('Date')['Bandwidth Usage'].sum()

# Print the total bandwidth usage for each date
for date, usage in usage_by_date.items():
    print(f"Date: {date.date()}, Total Bandwidth Usage: {usage}")

# Save the result to a new CSV file
usage_by_date.to_csv('total_bandwidth_usage_by_date.csv', header=True)

### 4. Remove unnecessary JSON fields and bugged strings from subuser names

Creates a list with needed fields and removes "klgg2" from oxyUsername and briefOxyUsername for each customer.

In [ ]:
import json

input_file = 'filtered_customers.json'
output_file = 'cleaned_customers.json'

# List of fields to keep
fields_to_keep = ['email', 'oxyId', 'oxyUsername', 'briefOxyId', 'briefOxyUsername']

# Read the input file
with open(input_file, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Remove "klgg2" from oxyUsername and briefOxyUsername for each customer
for customer in data:
    for field in ['oxyUsername', 'briefOxyUsername']:
        if field in customer and customer[field] is not None:
            customer[field] = customer[field].replace('klgg2', '')

# Extract the required fields for each customer
filtered_data = [{field: customer.get(field) for field in fields_to_keep} for customer in data]

# Write the filtered data to the output file
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(filtered_data, f, indent=4)

print(f"Filtered customer data saved to {output_file}.")

### 5. Parsing JSON and CSV files to data match between subuser and customer information using the fuzzywuzzy library

Utilizes the fuzzy stirng matching algorithm from the fuzzywuzzy library to match subusers extracted from subuser_bw_usage.csv with their corresponding customer data stored in cleaned_customers.json and generates the results into a new CSV file.

In [ ]:
import csv
import json
from fuzzywuzzy import fuzz

def find_matching_customer(subuser, customers):
    for customer in customers:
        if not isinstance(customer, dict):
            continue

        customer_email = customer.get('email', "")
        customer_oxy_username = customer.get('oxyUsername', "")
        customer_brief_oxy_username = customer.get('briefOxyUsername', "")

        if (fuzz.ratio(subuser, customer_oxy_username) >= 80) or (fuzz.ratio(subuser, customer_brief_oxy_username) >= 80) or (fuzz.ratio(subuser, customer_email.split("@")[0]) >= 80):
            if isinstance(customer.get('oxyId'), dict):
                subuser_id = customer['oxyId'].get('$numberInt', "") if customer_oxy_username == subuser else customer.get('briefOxyId', "")
            else:
                subuser_id = customer.get('oxyId', "") if customer_oxy_username == subuser else customer.get('briefOxyId', "")

            return {
                'email': customer_email,
                'subuser_type': 'Pro' if customer_oxy_username == subuser else 'Basic',
                'subuser_id': subuser_id or ""
            }
    return None

def main():
    subuser_file = 'subuser_bw_usage_sorted.csv'
    customer_file = 'cleaned_customers.json'
    output_file = 'final_subuser_bw_usage.csv'

    # Load subuser data
    subusers = []
    with open(subuser_file, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            subusers.append(row)

    # Load customer data
    with open(customer_file, 'r') as file:
        customers = json.load(file)

    # Match subusers with customers
    matched_data = []
    match_count = 0
    for subuser in subusers:
        subuser_name = subuser['Subuser']
        matching_customer = find_matching_customer(subuser_name, customers)
        if matching_customer is not None:
            matched_data.append({
                'Email': matching_customer['email'],
                'Subuser Type': matching_customer['subuser_type'],
                'Subuser ID': matching_customer['subuser_id'],
                'Subuser': subuser_name,
                'Date': subuser['Date'],
                'Bandwidth Usage': subuser['Bandwidth Usage']
            })
            match_count += 1
            print(f"Match found for Subuser: {subuser_name}")
        else:
            print(f"No match found for Subuser: {subuser_name}")

    # Write the matched data to the output file
    fieldnames = ['Email', 'Subuser Type', 'Subuser ID', 'Subuser', 'Date', 'Bandwidth Usage']
    with open(output_file, 'w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(matched_data)

    print("Matching completed. Output file generated.")
    print(f"Total matches found: {match_count} out of {len(subusers)} subusers.")

if __name__ == '__main__':
    main()